In [ ]:
# 8-Model Stack (two LR, two XGB, GB, RF, Linear SVM+Calib, KNN)
# Matches your existing cell style/format.
# Expects: X_tr, y_tr, X_va, y_va, X_te, y_te already defined.
# Uses your existing TwoStageStackTS class and prob_scorer.
# ============================================================

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import log_loss, brier_score_loss, roc_auc_score, make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.utils.validation import check_is_fitted
import numpy as np
import xgboost as xgb

# ============================================================================
# Custom Two-Stage Stacking Class
# ============================================================================
class TwoStageStackTS(BaseEstimator, ClassifierMixin):
    _estimator_type = "classifier"
    def __init__(self, base_estimators, meta_estimator, n_splits=5, gap=1, passthrough_idx=None):
        self.base_estimators = base_estimators
        self.meta_estimator = meta_estimator
        self.n_splits = n_splits
        self.gap = gap
        self.passthrough_idx = passthrough_idx

    def _oof(self, est, X, y, groups):
        splitter = PurgedGroupTimeSeriesSplit(
            n_splits=self.n_splits,
            group_gap=self.gap
        )
        oof = np.full(len(X), np.nan, float)
        for tr, te in splitter.split(X, y, groups=groups):
            if len(tr) == 0 or len(te) ==0:
                continue
            e = clone(est).fit(X[tr], y[tr])
            oof[te] = e.predict_proba(X[te])[:, 1]
        return oof

    def fit(self, X, y, groups=None):
        if groups is None:
            raise ValueError("TwoStageStackTS.fit requires 'groups' (e.g., normalized dates)")
        X = np.asarray(X); y = np.asarray(y)
        groups = np.asarray(groups)
        Zcols, self.base_fitted_ = [], []
        for _, est in self.base_estimators:
            Zcols.append(self._oof(est, X, y, groups))
        Z = np.column_stack(Zcols)
        mask = np.all(~np.isnan(Z), axis=1)
        Z_tr, y_tr = Z[mask], y[mask]
        if self.passthrough_idx is not None and len(self.passthrough_idx):
            Z_tr = np.hstack([Z_tr, X[mask][:, self.passthrough_idx]])
        self.meta_ = clone(self.meta_estimator).fit(Z_tr, y_tr)
        for name, est in self.base_estimators:
            self.base_fitted_.append((name, clone(est).fit(X, y)))
        self.mask_ = mask
        self.classes_ = np.array([0, 1])
        return self

    def _make_meta_X(self, X):
        cols = [est.predict_proba(X)[:, 1] for _, est in self.base_fitted_]
        Z = np.column_stack(cols)
        if self.passthrough_idx is not None and len(self.passthrough_idx):
            Z = np.hstack([Z, X[:, self.passthrough_idx]])
        return Z

    def predict_proba(self, X):
        check_is_fitted(self, ["meta_", "base_fitted_"])
        Z = self._make_meta_X(np.asarray(X))
        return self.meta_.predict_proba(Z)

    def predict(self, X):
        return (self.predict_proba(X)[:, 1] >= 0.5).astype(int)

# ----------------------------------------------------------------
# Scorer (same as before)
# ----------------------------------------------------------------
def probability_score(y_true, y_pred):
    return 0.6 * log_loss(y_true, y_pred) + 0.4 * brier_score_loss(y_true, y_pred)

prob_scorer = make_scorer(probability_score, response_method='predict_proba', greater_is_better=False)

tss = PurgedGroupTimeSeriesSplit(n_splits=5, group_gap=1)

# ============================================================================
# BASE MODEL 1: Logistic Regression (Elastic Net)
# ============================================================================
print("=" * 60)
print("Training Logistic Regression (Elastic Net)...")
print("=" * 60)

pipe_lr_elastic = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
    LogisticRegression(penalty="elasticnet", solver="saga", max_iter=10000, random_state=42)
)

lr_elastic_dist = {
    "logisticregression__C": np.logspace(-4, 3, 80),
    "logisticregression__l1_ratio": [0.1, 0.2, 0.3, 0.5, 0.7],
    "logisticregression__fit_intercept": [True, False],
    "logisticregression__class_weight": [None, "balanced"],
}

lr_elastic_search = RandomizedSearchCV(
    estimator=pipe_lr_elastic,
    param_distributions=lr_elastic_dist,
    n_iter=60,
    scoring=prob_scorer,
    cv=tss,
    n_jobs=-1,
    refit=True,
    random_state=42,
    verbose=1
)
lr_elastic_search.fit(X_tr, y_tr, groups=groups_tr)
print(f"Best Elastic Net LR params: {lr_elastic_search.best_params_}")
print(f"Best CV score: {-lr_elastic_search.best_score_:.4f}")

# ============================================================================
# BASE MODEL 2: Logistic Regression (L2/Ridge)
# ============================================================================
print("\n" + "=" * 60)
print("Training Logistic Regression (L2)...")
print("=" * 60)

pipe_lr_l2 = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
    LogisticRegression(penalty="l2", solver="lbfgs", max_iter=10000, random_state=42)
)

lr_l2_dist = {
    "logisticregression__C": np.logspace(-4, 4, 100),
    "logisticregression__fit_intercept": [True, False],
    "logisticregression__class_weight": [None, "balanced"],
}

lr_l2_search = RandomizedSearchCV(
    estimator=pipe_lr_l2,
    param_distributions=lr_l2_dist,
    n_iter=50,
    scoring=prob_scorer,
    cv=tss,
    n_jobs=-1,
    refit=True,
    random_state=42,
    verbose=1
)
lr_l2_search.fit(X_tr, y_tr, groups=groups_tr)
print(f"Best L2 LR params: {lr_l2_search.best_params_}")
print(f"Best CV score: {-lr_l2_search.best_score_:.4f}")

# ============================================================================
# BASE MODEL 3: XGBoost (Shallow / probability-oriented)
# ============================================================================
print("\n" + "=" * 60)
print("Training XGBoost (shallow)...")
print("=" * 60)

xgb_shallow = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    tree_method="hist",
    n_jobs=-1,
    random_state=42,
)

xgb_shallow_dist = {
    "max_depth": [3, 4],
    "learning_rate": [0.01, 0.02],
    "n_estimators": [300, 500],
    "min_child_weight": [3, 5, 7],
    "subsample": [0.7, 0.85, 0.95],
    "colsample_bytree": [0.6, 0.75, 0.9],
    "reg_lambda": [1.0, 2.0, 5.0],
    "reg_alpha": [0.0, 0.5, 1.0],
}

xgb_shallow_search = RandomizedSearchCV(
    estimator=xgb_shallow,
    param_distributions=xgb_shallow_dist,
    n_iter=60,
    scoring=prob_scorer,
    cv=tss,
    n_jobs=-1,
    refit=True,
    random_state=42,
    verbose=1
)
xgb_shallow_search.fit(X_tr, y_tr, groups=groups_tr)
print(f"Best XGB (shallow) params: {xgb_shallow_search.best_params_}")
print(f"Best CV score: {-xgb_shallow_search.best_score_:.4f}")

# ============================================================================
# BASE MODEL 4: XGBoost (Deeper / stronger)
# ============================================================================
print("\n" + "=" * 60)
print("Training XGBoost (deeper)...")
print("=" * 60)

xgb_deep = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    tree_method="hist",
    n_jobs=-1,
    random_state=42,
)

xgb_deep_dist = {
    "max_depth": [5],
    "learning_rate": [0.01, 0.015, 0.02],
    "n_estimators": [600, 800],
    "min_child_weight": [5, 7, 9],
    "subsample": [0.7, 0.85],
    "colsample_bytree": [0.6, 0.8],
    "reg_lambda": [2.0, 5.0, 8.0],
    "reg_alpha": [0.0, 0.5, 1.0],
}

xgb_deep_search = RandomizedSearchCV(
    estimator=xgb_deep,
    param_distributions=xgb_deep_dist,
    n_iter=50,
    scoring=prob_scorer,
    cv=tss,
    n_jobs=-1,
    refit=True,
    random_state=42,
    verbose=1
)
xgb_deep_search.fit(X_tr, y_tr, groups=groups_tr)
print(f"Best XGB (deeper) params: {xgb_deep_search.best_params_}")
print(f"Best CV score: {-xgb_deep_search.best_score_:.4f}")

# ============================================================================
# BASE MODEL 5: Gradient Boosting (sklearn)
# ============================================================================
print("\n" + "=" * 60)
print("Training Gradient Boosting (sklearn)...")
print("=" * 60)

gb_base = GradientBoostingClassifier(random_state=42)

gb_dist = {
    "learning_rate": [0.01, 0.02, 0.03],
    "n_estimators": [400, 600, 800],
    "max_depth": [3, 4],
    "min_samples_leaf": [10, 20],
    "subsample": [0.7, 0.85, 1.0],
}

gb_search = RandomizedSearchCV(
    estimator=gb_base,
    param_distributions=gb_dist,
    n_iter=60,
    scoring=prob_scorer,
    cv=tss,
    n_jobs=-1,
    refit=True,
    random_state=42,
    verbose=1
)
gb_search.fit(X_tr, y_tr, groups=groups_tr)
print(f"Best GB params: {gb_search.best_params_}")
print(f"Best CV score: {-gb_search.best_score_:.4f}")

# ============================================================================
# BASE MODEL 6: Random Forest (smoothed probs)
# ============================================================================
print("\n" + "=" * 60)
print("Training Random Forest...")
print("=" * 60)

rf_base = RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True)

rf_dist = {
    "n_estimators": [300, 400, 600],
    "max_depth": [8, 10, 12],
    "min_samples_leaf": [20, 30, 40],
    "min_samples_split": [20, 30],
    "max_features": ['sqrt', 0.5],
    "max_samples": [0.7, 0.85, None],
}

rf_search = RandomizedSearchCV(
    estimator=rf_base,
    param_distributions=rf_dist,
    n_iter=60,
    scoring=prob_scorer,
    cv=tss,
    n_jobs=-1,
    refit=True,
    random_state=42,
    verbose=1
)
rf_search.fit(X_tr, y_tr, groups=groups_tr)
print(f"Best RF params: {rf_search.best_params_}")
print(f"Best CV score: {-rf_search.best_score_:.4f}")

# ============================================================================
# BASE MODEL 7: Linear SVM (Calibrated)
# ============================================================================
print("\n" + "=" * 60)
print("Training Linear SVM (Calibrated)...")
print("=" * 60)

# Note: we wrap LinearSVC (no proba) in CalibratedClassifierCV for calibrated probabilities.
svm_calib = CalibratedClassifierCV(
    estimator=make_pipeline(
        StandardScaler(with_mean=True, with_std=True),
        LinearSVC(dual="auto", max_iter=5000, random_state=42)
    ),
    method="sigmoid",
    cv=3
)

svm_dist = {
    "estimator__linearsvc__C": np.logspace(-3, 2, 20),
    "method": ["sigmoid", "isotonic"],
}

svm_search = RandomizedSearchCV(
    estimator=svm_calib,
    param_distributions=svm_dist,
    n_iter=40,
    scoring=prob_scorer,
    cv=tss,
    n_jobs=-1,
    refit=True,
    random_state=42,
    verbose=1
)
svm_search.fit(X_tr, y_tr, groups=groups_tr)
print(f"Best SVM-Calibrated params: {svm_search.best_params_}")
print(f"Best CV score: {-svm_search.best_score_:.4f}")

# ============================================================================
# BASE MODEL 8: KNN (distance-weighted)
# ============================================================================
print("\n" + "=" * 60)
print("Training KNN (distance-weighted)...")
print("=" * 60)

pipe_knn = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
    KNeighborsClassifier(weights="distance")
)

knn_dist = {
    "kneighborsclassifier__n_neighbors": [25, 35, 50, 75, 100],
    "kneighborsclassifier__leaf_size": [20, 30, 40],
    "kneighborsclassifier__p": [1, 2],  # Manhattan / Euclidean
}

knn_search = RandomizedSearchCV(
    estimator=pipe_knn,
    param_distributions=knn_dist,
    n_iter=40,
    scoring=prob_scorer,
    cv=tss,
    n_jobs=-1,
    refit=True,
    random_state=42,
    verbose=1
)
knn_search.fit(X_tr, y_tr, groups=groups_tr)
print(f"Best KNN params: {knn_search.best_params_}")
print(f"Best CV score: {-knn_search.best_score_:.4f}")

# ============================================================================
# Rebuild Final Models with Best Hyperparameters
# ============================================================================
print("\n" + "=" * 60)
print("Building Final Base Models with Best Hyperparameters...")
print("=" * 60)

lr_elastic_final = make_pipeline(StandardScaler(), LogisticRegression(penalty="elasticnet", solver="saga", max_iter=10000, random_state=42))
lr_elastic_final.set_params(**lr_elastic_search.best_params_)

lr_l2_final = make_pipeline(StandardScaler(), LogisticRegression(penalty="l2", solver="lbfgs", max_iter=10000, random_state=42))
lr_l2_final.set_params(**lr_l2_search.best_params_)

xgb_shallow_final = CalibratedClassifierCV(
    estimator=xgb.XGBClassifier(objective="binary:logistic", eval_metric="logloss", tree_method="hist", n_jobs=-1, random_state=42, **xgb_shallow_search.best_params_),
    method='sigmoid',
    cv=3
) 
xgb_deep_final = CalibratedClassifierCV(
    estimator=xgb.XGBClassifier(objective="binary:logistic", eval_metric="logloss", tree_method="hist", n_jobs=-1, random_state=42, **xgb_deep_search.best_params_),
    method='sigmoid',
    cv=3
) 

gb_final = CalibratedClassifierCV(
    estimator=GradientBoostingClassifier(random_state=42, **gb_search.best_params_),
    method='sigmoid',
    cv=3
)

rf_final = CalibratedClassifierCV(
    estimator=RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True, **rf_search.best_params_),
    method='sigmoid',
    cv=3
)

# For SVM we reuse the fitted configuration (CalibratedClassifierCV with best params)
svm_final = CalibratedClassifierCV(
    estimator=make_pipeline(
        StandardScaler(),
        LinearSVC(dual="auto", max_iter=5000, random_state=42)
    ),
    method=svm_search.best_params_["method"],
    cv=3
)
# set best C on the inner LinearSVC
svm_final.estimator.set_params(linearsvc__C=svm_search.best_params_["estimator__linearsvc__C"])

knn_final = make_pipeline(StandardScaler(), KNeighborsClassifier(weights="distance"))
knn_final.set_params(**{
    k: v
    for k, v in knn_search.best_params_.items()
    if k.startswith("kneighborsclassifier__")
})

# ============================================================================
# Create Base Models List for Stacking
# ============================================================================
base_models = [
    ("lr_elastic",  lr_elastic_final),
    ("lr_l2",       lr_l2_final),
    ("xgb_shallow", xgb_shallow_final),
    ("xgb_deep",    xgb_deep_final),
    ("gb",          gb_final),
    ("rf",          rf_final),
    ("svm_linear",  svm_final),
    ("knn",         knn_final),
]

print("\nBase models ready for stacking!")
print(f"Total base models: {len(base_models)}")

# ============================================================================
# Meta-Learner (Elastic-Net Logistic on OOF)
# ============================================================================
print("\n" + "=" * 60)
print("Creating Meta-Learner...")
print("=" * 60)

meta = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
    LogisticRegression(
        solver="saga",
        penalty="elasticnet",
        l1_ratio=0.15,
        C=0.1,
        max_iter=5000,
        tol=1e-3,
        random_state=42
    )
)

# ============================================================================
# Fit Stacked Ensemble
# ============================================================================
stack = TwoStageStackTS(
    base_estimators=base_models,
    meta_estimator=meta,
    n_splits=5,
    gap=5,
    passthrough_idx=None
)

print("\nFitting stacked ensemble...")
stack.fit(X_tr, y_tr, groups=groups_tr)

# ============================================================================
# Evaluate
# ============================================================================
print("\n" + "=" * 60)
print("Validation Results")
print("=" * 60)
p_va = stack.predict_proba(X_va)[:, 1]
yhat = (p_va >= 0.5).astype(int)

print(f"Val Log Loss:  {log_loss(y_va, p_va):.4f}")
print(f"Val Brier:     {brier_score_loss(y_va, p_va):.4f}")
print(f"Val AUC:       {roc_auc_score(y_va, p_va):.4f}")
print(f"Val Accuracy:  {accuracy_score(y_va, yhat):.4f}")

from sklearn.calibration import calibration_curve
prob_true, prob_pred = calibration_curve(y_va, p_va, n_bins=10, strategy='quantile')
ece = np.mean(np.abs(prob_true - prob_pred))
print(f"Val ECE:       {ece:.4f}")

model = stack
print("\n✓ Model training complete!")
